<b>Note: This Jupyter Notebook is associated with the article [The Iron Condor Explained (and How to Trade The Options Strategy with Alpaca)](https://alpaca.markets/learn/iron-condor).</b>

# Step 1: Setting Up the Environment and Trade Parameters

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
!python3 -m pip install --upgrade alpaca-py

import pandas as pd
import numpy as np
from scipy.stats import norm
import alpaca
import time
from scipy.optimize import brentq
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, OptionLatestQuoteRequest, OptionChainRequest, OptionBarsRequest
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest,
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ClosePositionRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderStatus,
    OrderType,
    TimeInForce,
    QueryOrderStatus,
    ContractType,
)

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example (Please do not modify this. This example is for paper trading only).
PAPER = True

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

# Step 2: Choosing the Right Strike Price and Expiration and Many Other Factors

In [ ]:
# Select the underlying stock (Johnson & Johnson)
underlying_symbol = 'JNJ'

# Set the timezone
timezone = ZoneInfo('America/New_York')

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define a 15% range around the underlying price 
STRIKE_RANGE = 0.15

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.05

# Risk free rate for the options greeks and IV calculations
risk_free_rate = 0.01

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Set the open interest volume threshold
OI_THRESHOLD = 100

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Set the expiration date range for the options
min_expiration = today + timedelta(days=7)
max_expiration = today + timedelta(days=42)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Define a common expiration range for all legs between 14 days and 35 days
COMMON_EXPIRATION_RANGE = (14, 35)

# Each key corresponds to a leg and maps to a tuple of: (expiration range, IV range, delta range, theta range)
criteria = {
    'long_put':  (COMMON_EXPIRATION_RANGE, (0.15, 0.50), (-0.20, -0.01), (-0.1, -0.005)),
    'short_put': (COMMON_EXPIRATION_RANGE, (0.10, 0.40), (-0.30, -0.03), (-0.2, -0.01)),
    'short_call':(COMMON_EXPIRATION_RANGE, (0.10, 0.40), (0.03, 0.30), (-0.2, -0.01)),
    'long_call': (COMMON_EXPIRATION_RANGE, (0.15, 0.50), (0.01, 0.20), (-0.1, -0.005))
}

# Set target profit levels
TARGET_PROFIT_PERCENTAGE = 0.4

# Display the values
print(f"Underlying Symbol: {underlying_symbol}")
print(f"{underlying_symbol} price: {underlying_price}")
print(f"Strike Range: {STRIKE_RANGE}")
print(f"Buying Power Limit Percentage: {BUY_POWER_LIMIT}")
print(f"Risk Free Rate: {risk_free_rate}")
print(f"Account Buying Power: {buying_power}")
print(f"Buying Power Limit: {buying_power_limit}")
print(f"Open Interest Threshold: {OI_THRESHOLD}")
print(f"Minimum Expiration Date: {min_expiration}")
print(f"Maximum Expiration Date: {max_expiration}")
print(f"Minimum Strike Price: {min_strike}")
print(f"Maximum Strike Price: {max_strike}")


# Step 2: Calculations for Implied Volatility and Option Greeks with Black-Scholes model

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):
    
    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:
        
        print('Option price is close to intrinsic value; implied volatility is near zero.')
        return 0.0
    
    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None


def calculate_greeks(option_price, strike_price, expiration, underlying_price, risk_free_rate, option_type):
    T = (expiration - pd.Timestamp.now()).days / 365 # It is unconventional, but some use 225 days (# of annual trading days) in replace of 365 days
    T = max(T, 1e-6)  # Set minimum T to avoid zero
    
    if T == 1e-6:
        print('Option has expired or is expiring now; setting Greeks based on intrinsic value.')
        if option_type == 'put':
            delta = -1.0 if underlying_price < strike_price else 0.0
        else:
            delta = 1.0 if underlying_price > strike_price else 0.0
        gamma = 0.0
        theta = 0.0
        vega = 0.0
        return delta, gamma, theta, vega
    
    # Calculate IV
    IV = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)

    if IV is None or IV == 0.0:
        print('Implied volatility could not be determined, skipping Greek calculations.')
        return None
    
    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * IV ** 2) * T) / (IV * np.sqrt(T))
    d2 = d1 - IV * np.sqrt(T) # d2 for Theta calculation
    # Calculate Delta
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    # Calculate Gamma
    gamma = norm.pdf(d1) / (underlying_price * IV * np.sqrt(T))
    # Calculate Vega
    vega = underlying_price * np.sqrt(T) * norm.pdf(d1)
    # Calculate Theta
    if option_type == 'call':
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            - (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(d2))
        )
    else:
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            + (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(-d2))
        )
    # Convert annualized theta to daily theta
    theta /= 365
    
    return delta, gamma, theta, vega

# Step 4: Historical Market Data Analysis with its stock chart

## Stock Price Trends (Bar Chart Analysis)

In [ ]:
# Get the historical data for the underlying stock by symbol and timeframe
# ref. https://alpaca.markets/sdks/python/api_reference/data/option/historical.html
def get_stock_data(underlying_symbol, days=90):
    today = datetime.now(timezone).date()
    req = StockBarsRequest(
        symbol_or_symbols=[underlying_symbol],
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),     # specify timeframe
        start=today - timedelta(days=days),                          # specify start datetime, default=the beginning of the current day.
    )
    return stock_data_client.get_stock_bars(req).df

In [ ]:
priceData = get_stock_data(underlying_symbol, days=180)

# List of stock agg objects while dropping the symbol column
priceData = priceData.reset_index(level='symbol', drop=True)

import plotly.graph_objects as go

# Bar chart for the stock price
fig = go.Figure(data=[go.Candlestick(x=priceData.index,
                open=priceData['open'],
                high=priceData['high'],
                low=priceData['low'],
                close=priceData['close'])])

fig.show()


## Relative Volatility Index (RVI) for Volatility Directions and Its Degree

In [ ]:
def calculate_rvi(df, period=14):
    # Calculate daily price changes 
    df['price_change'] = df['close'].diff()

    # Separate up and down price changes
    df['up_change'] = df['price_change'].where(df['price_change'] > 0, 0)
    df['down_change'] = -df['price_change'].where(df['price_change'] < 0, 0)

    # Calculate std of up and down changes over the rolling period
    df['avg_up_std'] = df['up_change'].rolling(window=period).std()
    df['avg_down_std'] = df['down_change'].rolling(window=period).std()

    # Calculate RVI
    df['rvi'] = 100 - (100 / (1 + (df['avg_up_std'] / df['avg_down_std'])))

    # Smooth the RVI with a 4 periods moving average
    df['rvi_smoothed'] = df['rvi'].rolling(window=4).mean()

    return df['rvi_smoothed']

# smoothed RVI
calculate_rvi(priceData, period=21).iloc[30:]

## Bollinger Bands for Volatility Assessment

In [ ]:
# setup bollinger band calculations
def check_bb(df, period=14, multiplier=2):
    bollinger_bands = []
    # Calculate the Simple Moving Average (SMA)
    df['SMA'] = df['close'].rolling(window=period).mean()
    # Calculate the rolling standard deviation
    df['StdDev'] = df['close'].rolling(window=period).std()
    # Calculate the Upper Bollinger Band (two standard deviation)
    df['Upper Band'] = df['SMA'] + (multiplier * df['StdDev'])
    # Calculate the Lower Bollinger Band (two standard deviation)
    df['Lower Band'] = df['SMA'] - (multiplier * df['StdDev'])
    # Get the most recent Upper Band value
    upper_bollinger_band = df['Upper Band'].iloc[-1]
    lower_bollinger_band = df['Lower Band'].iloc[-1]
    
    bollinger_bands = [upper_bollinger_band, lower_bollinger_band]
    return bollinger_bands

bollinger_bands = check_bb(priceData, 14, 2)

# The current market price is not too close to the two-standard deviation level yet but is relatively closer to the higher Bollinger Band.
print(f"Latest Upper Bollinger Band is: {bollinger_bands[0]}. Latest Lower Bollinger Band is {bollinger_bands[1]}; while underlying stock '{underlying_symbol}' price is {underlying_price}.")


# Step 5: Setting Up For A Short Iron Condor Options Strategy (Find both put and call options)

In [ ]:
# Check for call options
def get_call_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration):
    
    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol],
        status=AssetStatus.ACTIVE,
        type=ContractType.CALL,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        expiration_date_gte=min_expiration,
        expiration_date_lte=max_expiration,
    )
    
    # Get call option chain of the underlying symbol
    call_options = trade_client.get_option_contracts(req).option_contracts
    return call_options

In [ ]:
# Check for put options
def get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration):
    
    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol],
        status=AssetStatus.ACTIVE,
        type=ContractType.PUT,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        expiration_date_gte=min_expiration,
        expiration_date_lte=max_expiration,        
    )
    
    # Get put option chain of the underlying symbol
    put_options = trade_client.get_option_contracts(req).option_contracts
    return put_options

In [ ]:
def validate_sufficient_OI(option_data, OI_THRESHOLD):
    '''Ensure that the option has the required fields and sufficient open interest.'''
    if option_data.open_interest is None or option_data.open_interest_date is None:
        return False
    if float(option_data.open_interest) <= OI_THRESHOLD:
        return False
    return True

In [ ]:
def build_option_dict(option_data, underlying_price, risk_free_rate):
    '''
    Helper to build an option dictionary from option_data and calculated metrics.
    '''
    # Retrieve the latest quote
    option_quote_req = OptionLatestQuoteRequest(symbol_or_symbols=option_data.symbol)
    option_quote = option_historical_data_client.get_option_latest_quote(option_quote_req)[option_data.symbol]
    option_price = (option_quote.bid_price + option_quote.ask_price) / 2

    strike_price = float(option_data.strike_price)
    expiration = pd.Timestamp(option_data.expiration_date)
    remaining_days = (expiration - pd.Timestamp.now()).days

    iv = calculate_implied_volatility(
        option_price=option_price,
        S=underlying_price,
        K=strike_price,
        T=max(remaining_days / 365, 1e-6),
        r=risk_free_rate,
        option_type=option_data.type.value
    )
    delta, _, theta, _ = calculate_greeks(
        option_price=option_price,
        strike_price=strike_price,
        expiration=expiration,
        underlying_price=underlying_price,
        risk_free_rate=risk_free_rate,
        option_type=option_data.type.value
    )

    # Build a candidate dictionary that can be augmented or returned
    candidate = {
        'id': option_data.id,
        'name': option_data.name,
        'symbol': option_data.symbol,
        'strike_price': option_data.strike_price,
        'root_symbol': option_data.root_symbol,
        'underlying_symbol': option_data.underlying_symbol,
        'underlying_asset_id': option_data.underlying_asset_id,
        'close_price': option_data.close_price,
        'close_price_date': option_data.close_price_date,
        'expiration': expiration,
        'remaining_days': remaining_days,
        'open_interest': option_data.open_interest,
        'open_interest_date': option_data.open_interest_date,
        'size': option_data.size,
        'status': option_data.status,
        'style': option_data.style,
        'tradable': option_data.tradable,
        'type': option_data.type,
        'initial_IV': iv,
        'initial_delta': delta,
        'initial_theta': theta,
        'initial_option_price': option_price,

    }
    return candidate

In [ ]:
def check_candidate_option_conditions(candidate, criteria, label):
    '''
    Check whether a candidate option meets the filtering criteria.
    The criteria is a tuple of (expiration_range, iv_range, delta_range, theta_range).
    '''
    expiration_range, iv_range, delta_range, theta_range = criteria

    if not (expiration_range[0] <= candidate['remaining_days'] <= expiration_range[1]):
        print(f"{candidate['symbol']} fails expiration condition for {label}.")
        return False
    if not (iv_range[0] <= candidate['initial_IV'] <= iv_range[1]):
        print(f"{candidate['symbol']} fails IV condition for {label}.")
        return False
    if not (delta_range[0] <= candidate['initial_delta'] <= delta_range[1]):
        print(f"{candidate['symbol']} fails delta condition for {label}.")
        return False
    if not (theta_range[0] <= candidate['initial_theta'] <= theta_range[1]):
        print(f"{candidate['symbol']} fails theta condition for {label}.")
        return False

    return True

In [ ]:
def pair_put_candidates(short_puts, long_puts, underlying_price):
    '''
    For the bull put spread, we require: long_put strike < short_put strike < underlying_price
    '''
    for short_put in short_puts:
        for long_put in long_puts:
            if long_put['strike_price'] < short_put['strike_price'] and short_put['strike_price'] < underlying_price and long_put['strike_price'] < underlying_price:
                print(f"Selected put spread: short_put {short_put['symbol']} and long_put {long_put['symbol']}.")
                return short_put, long_put
    return None, None


def pair_call_candidates(short_calls, long_calls, underlying_price):
    '''
    For the bear call spread, we require: underlying_price < short_call strike < long_call strike
    '''
    for short_call in short_calls:
        for long_call in long_calls:
            if short_call['strike_price'] < long_call['strike_price'] and short_call['strike_price'] > underlying_price and long_call['strike_price'] > underlying_price:
                print(f"Selected call spread: short_call {short_call['symbol']} and long_call {long_call['symbol']}.")
                return short_call, long_call
    return None, None

In [ ]:
def find_options_for_short_iron_condor(call_options, put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD):
    '''
    Modular workflow to find an iron condor with a single common expiration.
    Returns a list of legs in the order:
       [short_put, long_put, short_call, long_call]
    '''
    common_expirations = None

    # Instead of immediately enforcing common expiration, store candidates by expiration date
    put_candidates_by_expiration = {}
    call_candidates_by_expiration = {}

    # Process put options
    for option_data in put_options:
        # If option does not meet the Open Interest Threshold, skip the process
        if not validate_sufficient_OI(option_data, OI_THRESHOLD):
            continue
        candidate = build_option_dict(option_data, underlying_price, risk_free_rate)
        
        # For puts, consider only options that are OTM (strike below underlying)
        if candidate['strike_price'] >= underlying_price:
            print(f"Skipping {candidate['symbol']}: not OTM for puts (strike {candidate['strike_price']} >= underlying {underlying_price}).")
            continue
        
        expiration = candidate['expiration']
        if expiration not in put_candidates_by_expiration:
            put_candidates_by_expiration[expiration] = {'long_put': [], 'short_put': []}
        
        # Add an option candidate to dictionary if it meets the criteria for 'long put'
        if check_candidate_option_conditions(candidate, criteria['long_put'], 'long_put'):
            put_candidates_by_expiration[expiration]['long_put'].append(candidate)
            print(f"Added {candidate['symbol']} as a long put candidate for expiration {expiration}.")

        # Add an option candidate to dictionary if it meets the criteria for 'short put'
        if check_candidate_option_conditions(candidate, criteria['short_put'], 'short_put'):
            put_candidates_by_expiration[expiration]['short_put'].append(candidate)
            print(f"Added {candidate['symbol']} as a short put candidate for expiration {expiration}.")

    # Process call options
    for option_data in call_options:
        if not validate_sufficient_OI(option_data, OI_THRESHOLD):
            continue
        candidate = build_option_dict(option_data, underlying_price, risk_free_rate)
        
        # For calls, consider only options that are OTM (strike above underlying)
        if candidate['strike_price'] <= underlying_price:
            print(f"Skipping {candidate['symbol']}: not OTM for calls (strike {candidate['strike_price']} <= underlying {underlying_price}).")
            continue
        
        expiration = candidate['expiration']
        if expiration not in call_candidates_by_expiration:
            call_candidates_by_expiration[expiration] = {'short_call': [], 'long_call': []}
        
        if check_candidate_option_conditions(candidate, criteria['short_call'], 'short_call'):
            call_candidates_by_expiration[expiration]['short_call'].append(candidate)
            print(f"Added {candidate['symbol']} as a short call candidate for expiration {expiration}.")
        if check_candidate_option_conditions(candidate, criteria['long_call'], 'long_call'):
            call_candidates_by_expiration[expiration]['long_call'].append(candidate)
            print(f"Added {candidate['symbol']} as a long call candidate for expiration {expiration}.")

    # Find common expiration(s) present in both puts and calls
    common_expirations = set(put_candidates_by_expiration.keys()).intersection(set(call_candidates_by_expiration.keys()))
    
    if not common_expirations:
        raise Exception('No common expiration found across put and call candidates.')

    # Choose the expiration with the most candidates
    selected_expiration = max(common_expirations, key=lambda exp: (len(put_candidates_by_expiration[exp]['long_put']) +
                                                        len(put_candidates_by_expiration[exp]['short_put']) +
                                                        len(call_candidates_by_expiration[exp]['short_call']) +
                                                        len(call_candidates_by_expiration[exp]['long_call'])))
    print(f"Selected common expiration: {selected_expiration}")

    # Use candidates only from the selected common expiration
    puts_for_selected_expiration = put_candidates_by_expiration[selected_expiration]
    calls_for_selected_expiration = call_candidates_by_expiration[selected_expiration]

    # Pair up the put candidates
    chosen_short_put, chosen_long_put = pair_put_candidates(puts_for_selected_expiration['short_put'], puts_for_selected_expiration['long_put'], underlying_price)
    # Pair up the call candidates
    chosen_short_call, chosen_long_call = pair_call_candidates(calls_for_selected_expiration['short_call'], calls_for_selected_expiration['long_call'], underlying_price)

    if not (chosen_short_put and chosen_long_put and chosen_short_call and chosen_long_call):
        raise Exception('Could not find a valid combination for the iron condor for the selected expiration.')

    # Check buying power requirements
    # For puts: maximum risk = (short_put strike - long_put strike) * option size
    # For calls: maximum risk = (long_call strike - short_call strike) * option size
    option_size = float(chosen_short_put['size'])
    risk_put = (chosen_short_put['strike_price'] - chosen_long_put['strike_price']) * option_size
    risk_call = (chosen_long_call['strike_price'] - chosen_short_call['strike_price']) * option_size

    print(f"Calculated bull put spread risk: {risk_put}, bear call spread risk: {risk_call}.")
    if risk_put >= buying_power_limit or risk_call >= buying_power_limit:
        raise Exception('Buying power limit exceeded for the iron condor risk.')

    # Return the four legs in the following order
    iron_condor = [chosen_short_put, chosen_long_put, chosen_short_call, chosen_long_call]
    print('\nReturning iron condor legs:')
    for leg in iron_condor:
        print(f"{leg['symbol']} at strike {leg['strike_price']}")
    return iron_condor

In [ ]:
call_options = get_call_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)
put_options = get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)
iron_condor_order_legs = find_options_for_short_iron_condor(call_options, put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD)

# Step 6: Executing A Short Iron Condor Options Strategy

In [ ]:
# iron_condor_order_legs = find_options_for_short_iron_condor(call_options, put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD)

# # Place orders for the iron condor spread if all options are found
if iron_condor_order_legs:
    # Create a list for the order request
    order_legs = []
    ## Append short put
    order_legs.append(OptionLegRequest(
        symbol=iron_condor_order_legs[0]["symbol"],
        side=OrderSide.SELL,
        ratio_qty=1
    ))
    ## Append long put
    order_legs.append(OptionLegRequest(
        symbol=iron_condor_order_legs[1]["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))
    ## Append short call
    order_legs.append(OptionLegRequest(
        symbol=iron_condor_order_legs[2]["symbol"],
        side=OrderSide.SELL,
        ratio_qty=1
    ))
    ## Append long call
    order_legs.append(OptionLegRequest(
        symbol=iron_condor_order_legs[3]["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))

    # Place the order for both legs simultaneously
    req = MarketOrderRequest(
        qty=1,
        order_class=OrderClass.MLEG,
        time_in_force=TimeInForce.DAY,
        legs=order_legs
    )
    res = trade_client.submit_order(req)
    print("Short Iron Condor order placed successfully.")

# Cancel Multi-leg Orders (Optional)

You can cancel the order only when the order has not be filled.

In [ ]:
# Query by the order's id
q1 = trade_client.get_order_by_client_id(res.client_order_id)

# Replace overall order
if q1.status != OrderStatus.FILLED:
    # Cancel the whole order
    trade_client.cancel_order_by_id(res.id)
    print(f"Canceled order: {res}")

else:
    print("Order is already filled.")

In [ ]:
# Initialize the Option Historical Data Client
option_historical_data_client = OptionHistoricalDataClient(
    api_key=API_KEY, 
    secret_key=API_SECRET, 
    url_override=BASE_URL
)

# Define the request parameters
req = OptionBarsRequest(
    #  iron_condor_order_legs[1]["symbol"] = the short put option
    symbol_or_symbols=iron_condor_order_legs[1]["symbol"],
    timeframe=TimeFrame.Day,  # Choose timeframe (Minute, Hour, Day, etc.)
    start="2025-01-01",  # Start date
    end="2025-02-21"  # End date
)

option_historical_data_client.get_option_bars(req)
